In [1]:
#!/usr/bin/env python
# coding: utf-8

import os
import sys
import platform
import logging
import argparse
import trino
import io
import boto3
from itertools import islice
from datetime import datetime, date, timedelta
import pendulum
import pandas as pd
import numpy as np


# Add parent directory to path
repo_path = '/Users/apple/Documents/naarni/repo/dview-naarni-data-platform'
sys.path.append(os.path.join(repo_path, 'tasks'))

# Import necessary files and its respective functions
from common.db_operations import connect_to_trino, fetch_data_for_day, write_df_to_iceberg,drop_table,execute_query
from common.optimizer_logic import optimize_dataframe_memory

# Import business logic functions
from biz_logic.energy_mileage.energy_mileage_daily_v0 import energy_mileage_stats ,impute_odometer_readings

from biz_logic.energy_consumption.energy_consumption_report import energy_consumption_stats

# Configure basic logging
logging.basicConfig(level=logging.INFO,
                    format='%(asctime)s - %(levelname)s - %(message)s',
                    datefmt='%Y-%m-%d %H:%M:%S')

# Print the Python version being used
print(f"Using Python version: {platform.python_version()}")

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

Using Python version: 3.13.7


In [2]:
# ---- reporting config (edit ONLY this) ----
TABLE_NAME = "can_parsed_output_100"   # <— change only this

# derived (don’t edit)
REPORT_TABLE = f"adhoc.facts_prod.{TABLE_NAME}"
REPORT_S3_LOCATION = f"s3a://naarni-data-lake/aqua/warehouse/facts_prod.db/{TABLE_NAME}/"

In [10]:
# Connect to Trino
conn = connect_to_trino(host="analytics.internal.naarni.com",port=443,user="admin",catalog="adhoc",schema="default")

query = """select 
    id, date(timestamp),odometerreading, brakepedalpos, bat_voltage,cell_min_voltage
from 
    clickhouse.naarni.can_parsed_output_100
where 
  id =cast('3' as varbinary)
  and date(timestamp) = date('2025-10-10')"""    
# Execute queries and fetch data

# Fetch cpo100 data
df = execute_query(conn,query)

display(df.head())
conn.close()

2025-10-28 15:21:39 - INFO - 🔌 STEP 1: Connecting to Trino...
2025-10-28 15:21:39 - INFO - ✅ STEP 1: Connected to Trino
2025-10-28 15:21:39 - INFO - ⚙️ Executing query...
2025-10-28 15:21:42 - INFO - ✅ Query executed successfully with results!


,id,_col1,odometerreading,brakepedalpos,bat_voltage,cell_min_voltage
0,b'3',2025-10-10,NaN,0.0,635.9,3.308
1,b'3',2025-10-10,29766.0,0.0,634.9,3.305
2,b'3',2025-10-10,29766.0,0.0,635.4,3.308
3,b'3',2025-10-10,NaN,0.0,636.4,3.313
4,b'3',2025-10-10,29766.0,0.0,637.9,3.321


In [4]:
def get_all_battery_data(start_date, end_date, vehicle_ids):
    """
    Fetch all battery data for the specified date range and vehicle IDs in one go.
    
    Args:
        start_date: Start date in 'YYYY-MM-DD' format
        end_date: End date in 'YYYY-MM-DD' format
        vehicle_ids: List of vehicle IDs
        
    Returns:
        DataFrame with battery data
    """
    logging.info(f"Fetching all battery data from {start_date} to {end_date} for vehicles {vehicle_ids}")
    
    # Format vehicle IDs for the query
    vehicle_ids_str = ', '.join([f"'{vid}'" for vid in vehicle_ids])
    
    # Connect to Trino
    conn = connect_to_trino(host="analytics.internal.naarni.com",port=443,user="admin",catalog="adhoc",schema="default")

    # Query for cpo100 data
    cpo100_query = f"""
    SELECT 
        id, CAST(timestamp AS TIMESTAMP) AT TIME ZONE 'Asia/Kolkata' as timestamp, dt, 
        batterycoolingstate, batterycoolanttemperature,
        vehiclereadycondition, gun_connection_status, ignitionstatus, vehicle_speed_vcu, gear_position,
-- Battery Temperature parameters
        pack1_cellmax_temperature, pack1_maxtemperature_cell_number, pack1_cell_min_temperature, pack1_celltemperature_cellnumber,
-- GUN A and B Temperature parameters
        guna_dcp_temperature, guna_dcm_temperature, gunb_dcp_temperature, gunb_dcm_temperature,
-- Battery Voltage parameters
        bat_voltage, cellmax_voltagecellnumber, cell_max_voltage, cellminvoltagecellnumber, cell_min_voltage, dcdcbusvoltage,
-- Alarms
        temperaturedifferencealarm, chargingcurrentalarm, dischargecurrentalarm,
-- Fault codes
        lowpressureoilpumpfaultcode, bms_fault_code, vcu_fault_code, fiveinone_faultcode
    FROM 
        facts_prod.can_parsed_output_100
    WHERE 
        id IN ({vehicle_ids_str})
        AND DATE(timestamp AT TIME ZONE 'Asia/Kolkata') >= DATE('{start_date}')
        AND DATE(timestamp AT TIME ZONE 'Asia/Kolkata') <= DATE('{end_date}')
    """

    # Query for can_ac data
    charging_data_query = f"""
    SELECT 
        *
    FROM 
        facts_prod.energy_consumption_report
    WHERE
        vehicle_id IN ({vehicle_ids_str})
        AND DATE(start_time)  >= DATE('{start_date}')
        AND DATE(end_time) <= DATE('{end_date}')
    """

    # Execute queries and fetch data
    cur = conn.cursor()

    # Fetch cpo100 data
    cur.execute(cpo100_query)
    cpo100_columns = [desc[0] for desc in cur.description]
    cpo100_rows = cur.fetchall()
    df_cpo100 = pd.DataFrame(cpo100_rows, columns=cpo100_columns)

    logging.info(f"Done Fetching cpo100 data.")
    logging.info(f"Retrieved {len(df_cpo100)} cpo100 records")    

    # Fetch charging data
    cur.execute(charging_data_query)
    charging_data_columns = [desc[0] for desc in cur.description]
    charging_data_rows = cur.fetchall()
    df_charging_data = pd.DataFrame(charging_data_rows, columns=charging_data_columns)

    logging.info(f"Done Fetching charging data.")
    logging.info(f"Retrieved {len(df_charging_data)} charging records")
    # Close connections
    cur.close()
    conn.close()

    # Convert timestamp columns to datetime if they aren't already
    df_cpo100['timestamp'] = pd.to_datetime(df_cpo100['timestamp'])
    df_charging_data['start_time'] = pd.to_datetime(df_charging_data['start_time'])
    df_charging_data['end_time'] = pd.to_datetime(df_charging_data['end_time'])

    return df_cpo100, df_charging_data

In [5]:
date_str = "2025-10-20"  # Date for which data is to be processed

# Parse the date string as a date object
target_date = datetime.strptime(date_str, "%Y-%m-%d").date()

# Create datetime objects for the start and end of the day in IST
ist_start = datetime.combine(target_date, datetime.min.time())
ist_end = ist_start + timedelta(days=2)

# Convert IST to UTC for the database query
# IST is UTC+5:30, so we subtract 5 hours and 30 minutes
utc_start = ist_start - timedelta(hours=5, minutes=30)
utc_end = ist_end - timedelta(hours=5, minutes=30)

logging.info(f"🔍 Query window (UTC): {utc_start} → {utc_end}")
logging.info(f"🔍 Query window (IST): {ist_start} → {ist_end}")

2025-10-28 15:05:26 - INFO - 🔍 Query window (UTC): 2025-10-19 18:30:00 → 2025-10-21 18:30:00
2025-10-28 15:05:26 - INFO - 🔍 Query window (IST): 2025-10-20 00:00:00 → 2025-10-22 00:00:00


In [6]:
vehicle_ids = ['16']
start_date = "2025-10-20"  # Date for which data is to be processed
end_date = "2025-10-21"  # Date for which data is to be processed

can_df, charging_df = get_all_battery_data(start_date, end_date, vehicle_ids)

2025-10-28 15:05:26 - INFO - Fetching all battery data from 2025-10-20 to 2025-10-21 for vehicles ['16']
2025-10-28 15:05:26 - INFO - 🔌 STEP 1: Connecting to Trino...
2025-10-28 15:05:26 - INFO - ✅ STEP 1: Connected to Trino
2025-10-28 15:10:27 - INFO - Done Fetching cpo100 data.
2025-10-28 15:10:27 - INFO - Retrieved 96361 cpo100 records
2025-10-28 15:10:28 - INFO - Done Fetching charging data.
2025-10-28 15:10:28 - INFO - Retrieved 4 charging records


In [7]:
display(can_df.head())
display(charging_df.head())

,id,timestamp,dt,batterycoolingstate,batterycoolanttemperature,vehiclereadycondition,gun_connection_status,ignitionstatus,vehicle_speed_vcu,gear_position,pack1_cellmax_temperature,pack1_maxtemperature_cell_number,pack1_cell_min_temperature,pack1_celltemperature_cellnumber,guna_dcp_temperature,guna_dcm_temperature,gunb_dcp_temperature,gunb_dcm_temperature,bat_voltage,cellmax_voltagecellnumber,cell_max_voltage,cellminvoltagecellnumber,cell_min_voltage,dcdcbusvoltage,temperaturedifferencealarm,chargingcurrentalarm,dischargecurrentalarm,lowpressureoilpumpfaultcode,bms_fault_code,vcu_fault_code,fiveinone_faultcode
0,16,2025-10-21 12:54:02.829000+05:30,2025-10-21,None,None,0.0,1.0,0.0,0.0,0.0,33.0,80.0,28.0,9.0,37.0,37.0,43.0,43.0,647.7,480.0,3.384,88.0,3.368,NaN,0.0,0.0,0.0,NaN,0.0,0.0,None
1,16,2025-10-21 12:54:07.269000+05:30,2025-10-21,None,None,0.0,1.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,647.7,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,NaN,0.0,0.0,None
2,16,2025-10-21 12:51:44.649000+05:30,2025-10-21,None,None,0.0,1.0,0.0,0.0,0.0,33.0,80.0,28.0,9.0,37.0,37.0,43.0,43.0,647.2,480.0,3.380,88.0,3.366,NaN,0.0,0.0,0.0,NaN,0.0,0.0,None
3,16,2025-10-21 12:51:45.669000+05:30,2025-10-21,None,None,0.0,1.0,0.0,0.0,0.0,33.0,80.0,28.0,9.0,37.0,37.0,43.0,43.0,647.2,480.0,3.381,88.0,3.366,648.0,0.0,0.0,0.0,NaN,0.0,0.0,None
4,16,2025-10-21 12:51:46.829000+05:30,2025-10-21,None,None,0.0,1.0,0.0,0.0,0.0,33.0,80.0,28.0,9.0,37.0,37.0,43.0,43.0,647.2,480.0,3.381,88.0,3.366,648.0,0.0,0.0,0.0,NaN,0.0,0.0,None


,vehicle_id,start_time,end_time,charge_dur,soc_start,soc_end,tpc_kwh,soc_kwh,diff_kwh_percent
0,16,2025-10-21 18:04:06.990,2025-10-21 19:37:00.129,5572,26.8,100.0,299.45,309.64,3.35
1,16,2025-10-21 08:03:11.248,2025-10-21 09:18:21.328,4502,35.6,98.8,280.72,267.34,4.88
2,16,2025-10-21 07:05:29.408,2025-10-21 07:08:34.528,5,35.6,35.6,0.00,0.00,0.00
3,16,2025-10-21 14:23:01.405,2025-10-21 14:39:59.762,1016,53.6,58.0,19.01,18.61,2.10


In [8]:
can_df.head().to_clipboard()

In [9]:
import pytz

# --- Define IST timezone ---
ist = pytz.timezone("Asia/Kolkata")

# --- Ensure timestamp columns exist ---
if 'timestamp' in can_df.columns:
    can_df['timestamp'] = pd.to_datetime(can_df['timestamp'], errors='coerce')

# --- Ensure start_time and end_time parsed properly ---
charging_df['start_time'] = pd.to_datetime(charging_df['start_time'], errors='coerce')
charging_df['end_time'] = pd.to_datetime(charging_df['end_time'], errors='coerce')

# --- Normalize timezone awareness ---
# If timestamps are naive, localize to IST; if they’re already aware, convert to IST
def ensure_ist(df, col):
    if df[col].dt.tz is None:
        df[col] = df[col].dt.tz_localize('Asia/Kolkata')
    else:
        df[col] = df[col].dt.tz_convert('Asia/Kolkata')

ensure_ist(can_df, 'timestamp')
ensure_ist(charging_df, 'start_time')
ensure_ist(charging_df, 'end_time')

# --- Parameter groups ---
gun_temp_cols = [
    "guna_dcp_temperature", "guna_dcm_temperature",
    "gunb_dcp_temperature", "gunb_dcm_temperature"
]

battery_temp_cols = [
    "pack1_cellmax_temperature", "pack1_maxtemperature_cell_number",
    "pack1_cell_min_temperature", "pack1_celltemperature_cellnumber",
    "batterycoolanttemperature", "motor_temperature",
    "igbt_temperature", "air_compreesor_temperature"
]

battery_voltage_cols = [
    "dcdcbusvoltage", "bat_voltage",
    "cellmax_voltagecellnumber", "cell_max_voltage",
    "cellminvoltagecellnumber", "cell_min_voltage"
]

target_cols = gun_temp_cols + battery_temp_cols + battery_voltage_cols
available_cols = [col for col in target_cols if col in can_df.columns]
can_df = can_df[['timestamp'] + available_cols]

# --- Compute per-session stats ---
def calc_param_stats(df, start, end):
    # Filter data within the time window
    mask = (df['timestamp'] >= start) & (df['timestamp'] <= end)
    subset = df.loc[mask]

    stats = {}
    if subset.empty:
        # No CAN data found for this session → fill with NaN
        for col in available_cols:
            stats[f'{col}_mean'] = np.nan
            # stats[f'{col}_median'] = np.nan
            stats[f'{col}_min'] = np.nan
            stats[f'{col}_max'] = np.nan
        return stats

    # Compute safely
    for col in available_cols:
        col_data = subset[col].dropna()
        if len(col_data) == 0:
            stats[f'{col}_mean'] = np.nan
            # stats[f'{col}_median'] = np.nan
            stats[f'{col}_min'] = np.nan
            stats[f'{col}_max'] = np.nan
        else:
            stats[f'{col}_mean'] = col_data.mean().round(1)
            # stats[f'{col}_median'] = col_data.median()
            stats[f'{col}_min'] = col_data.min()
            stats[f'{col}_max'] = col_data.max()
    return stats


# --- Iterate sessions ---
charging_stats = []
for _, row in charging_df.iterrows():
    session_stats = {
        "vehicle_id": row.get("vehicle_id", np.nan),
        "start_time": row["start_time"],
        "end_time": row["end_time"],
        "charge_dur_mins": row.get("charge_dur", np.nan),
        "soc_start": row.get("soc_start", np.nan),
        "soc_end": row.get("soc_end", np.nan),
        "energy_consumed_kWh": row.get("tpc_kwh", np.nan),
    }
    session_stats.update(calc_param_stats(can_df, row["start_time"], row["end_time"]))
    charging_stats.append(session_stats)

charging_analysis_df = pd.DataFrame(charging_stats)

# --- Display results ---
print("✅ Timezone consistency applied successfully")
display(charging_analysis_df.head())

✅ Timezone consistency applied successfully


,vehicle_id,start_time,end_time,charge_dur_mins,soc_start,soc_end,energy_consumed_kWh,guna_dcp_temperature_mean,guna_dcp_temperature_min,guna_dcp_temperature_max,guna_dcm_temperature_mean,guna_dcm_temperature_min,guna_dcm_temperature_max,gunb_dcp_temperature_mean,gunb_dcp_temperature_min,gunb_dcp_temperature_max,gunb_dcm_temperature_mean,gunb_dcm_temperature_min,gunb_dcm_temperature_max,pack1_cellmax_temperature_mean,pack1_cellmax_temperature_min,pack1_cellmax_temperature_max,pack1_maxtemperature_cell_number_mean,pack1_maxtemperature_cell_number_min,pack1_maxtemperature_cell_number_max,pack1_cell_min_temperature_mean,pack1_cell_min_temperature_min,pack1_cell_min_temperature_max,pack1_celltemperature_cellnumber_mean,pack1_celltemperature_cellnumber_min,pack1_celltemperature_cellnumber_max,batterycoolanttemperature_mean,batterycoolanttemperature_min,batterycoolanttemperature_max,dcdcbusvoltage_mean,dcdcbusvoltage_min,dcdcbusvoltage_max,bat_voltage_mean,bat_voltage_min,bat_voltage_max,cellmax_voltagecellnumber_mean,cellmax_voltagecellnumber_min,cellmax_voltagecellnumber_max,cell_max_voltage_mean,cell_max_voltage_min,cell_max_voltage_max,cellminvoltagecellnumber_mean,cellminvoltagecellnumber_min,cellminvoltagecellnumber_max,cell_min_voltage_mean,cell_min_voltage_min,cell_min_voltage_max
0,16,2025-10-21 18:04:06.990000+05:30,2025-10-21 19:37:00.129000+05:30,5572,26.8,100.0,299.45,30.4,-40.0,31.0,30.2,-40.0,31.0,30.4,-40.0,31.0,30.4,-40.0,31.0,31.0,-40.0,32.0,80.9,0.0,101.0,27.0,-40.0,28.0,9.3,0.0,16.0,NaN,NaN,NaN,628.9,4.0,643.0,632.2,0.0,642.7,244.1,0.0,575.0,3.3,0.0,3.352,388.9,0.0,575.0,3.3,0.0,3.345
1,16,2025-10-21 08:03:11.248000+05:30,2025-10-21 09:18:21.328000+05:30,4502,35.6,98.8,280.72,40.1,-40.0,41.0,39.8,-40.0,40.0,40.3,-40.0,44.0,40.3,-40.0,44.0,34.6,-40.0,37.0,80.0,0.0,100.0,31.6,-40.0,34.0,8.9,0.0,9.0,NaN,NaN,NaN,623.7,50.0,640.0,628.9,0.0,640.0,248.8,0.0,575.0,3.3,0.0,3.340,257.7,0.0,529.0,3.3,0.0,3.332
2,16,2025-10-21 07:05:29.408000+05:30,2025-10-21 07:08:34.528000+05:30,5,35.6,35.6,0.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,16,2025-10-21 14:23:01.405000+05:30,2025-10-21 14:39:59.762000+05:30,1016,53.6,58.0,19.01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
